In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import OUTPUT_PATH, Dep, DepReport
from typing import Dict
import pickle


with open(OUTPUT_PATH / "bcc.pkl", "rb") as f:
    data: Dict[str, Dict[Dep, DepReport]] = pickle.load(f)

data.keys()

dict_keys(['bashreadline', 'bindsnoop', 'biolatency', 'biopattern', 'biosnoop', 'biostacks', 'biotop', 'bitesize', 'cachestat', 'capable', 'cpudist', 'cpufreq', 'drsnoop', 'execsnoop', 'exitsnoop', 'filelife', 'filetop', 'fsdist', 'fsslower', 'funclatency', 'futexctn', 'gethostlatency', 'hardirqs', 'javagc', 'klockstat', 'ksnoop', 'llcstat', 'mdflush', 'memleak', 'mountsnoop', 'numamove', 'offcputime', 'oomkill', 'opensnoop', 'profile', 'readahead', 'runqlat', 'runqlen', 'runqslower', 'sigsnoop', 'slabratetop', 'softirqs', 'solisten', 'statsnoop', 'syncsnoop', 'syscount', 'tcpconnect', 'tcpconnlat', 'tcplife', 'tcppktlat', 'tcprtt', 'tcpstates', 'tcpsynbl', 'tcptop', 'tcptracer', 'vfsstat', 'wakeuptime'])

In [3]:
data["Summary"] = {
    dep: report for deps in data.values() for dep, report in deps.items()
}

In [62]:
from depsurf import IssueEnum, DepKind, TAB_PATH
from collections import defaultdict
import pandas as pd

results = {}


for prog_name, deps in data.items():
    if prog_name in (
        "bashreadline",
        "ksnoop",
        "javagc",
        "gethostlatency",
        "funclatency",
    ):
        continue

    result = defaultdict(int)
    for dep, report in deps.items():
        kind = dep.kind
        result[(kind, "Count")] += 1

        num_absent = sum(
            1 for e in report.status.values() if IssueEnum.ABSENT in e.enums
        )
        result[(kind, IssueEnum.ABSENT)] += min(num_absent, 1)

        num_delta = sum(1 for e in report.delta.values() if e.changes)
        result[(kind, IssueEnum.CHANGE)] += min(num_delta, 1)

        if kind == DepKind.FUNC:
            for issue in [
                IssueEnum.PARTIAL_INLINE,
                IssueEnum.FULL_INLINE,
                IssueEnum.RENAME,
                IssueEnum.DUPLICATE,
            ]:
                num_issue = sum(1 for e in report.status.values() if issue in e.enums)
                result[(kind, issue)] += min(num_issue, 1)

    results[prog_name] = result

df = pd.DataFrame(results).T.fillna(0).astype(int)

column_order = [
    DepKind.FUNC,
    DepKind.STRUCT,
    DepKind.FIELD,
    DepKind.TRACEPOINT,
    DepKind.SYSCALL,
]
df = df.reindex(sorted(df.columns, key=lambda x: column_order.index(x[0])), axis=1)


def rename_cols(col):
    kind, issue = col
    if kind == DepKind.TRACEPOINT:
        kind = "Tracept"
    if kind == DepKind.SYSCALL:
        kind = "Syscall"
    return (kind, issue)


df.columns = df.columns.map(rename_cols)
df = df.sort_values(by=df.columns.tolist(), ascending=False)
df_non_summary = df.loc[df.index != "Summary"]
df = pd.concat([df_non_summary, df.loc[["Summary"]]])
df.loc["No. Tools"] = df_non_summary.map(lambda x: min(x, 1)).sum(axis=0)
# df.loc["Pct. Tools"] = (df.loc["No. Tools"] / len(df_non_summary) * 100).astype(int)


df_latex = df.copy()
df_latex = df_latex.replace(0, r"\color{lightgray}{-}")
df_latex.columns = df_latex.columns.map(lambda x: (x[0], f"\\head{{{x[1]}}}"))
with open(TAB_PATH / "bcc.tex", "w") as f:
    latex = df_latex.to_latex(
        multicolumn_format="c|",
        column_format="l|rrrrrrr|rrr|rrr|rrr|rrr",
    )
    latex = latex.replace(r"{c|}{Syscall}", r"{c}{Syscall}")
    latex = latex.replace(r"Summary", r"\midrule No. Deps")
    f.write(latex)